In [ ]:
import requests
from requests.adapters import HTTPAdapter, Retry
import json
import re
import sys
import pandas as pd

In [ ]:
url = "https://rest.uniprot.org/uniprotkb/search?format=json&query=%28%28fragment%3Afalse%29+AND+%28taxonomy_id%3A2759%29+AND+%28length%3A%5B40+TO+*%5D%29+AND+%28reviewed%3Atrue%29+AND+%28existence%3A1%29+AND+%28ft_signal_exp%3A*%29%29&size=500"
response = requests.get(url)
session = requests.Session()

In [ ]:
def get_next_link(headers):
    if "Link" in headers:
        links = headers["Link"].split(",")
        for link in links:
            match = re.search(r'<(.+)>; rel="next"', link)
            if match:
                return match.group(1)
    return None

def get_batch(batch_url):
    while batch_url:
        response = session.get(batch_url)
        response.raise_for_status()
        total = response.headers.get("x-total-results")
        yield response, total
        batch_url = get_next_link(response.headers)

In [ ]:
def get_kingdom(entry):
  if "Fungi" in entry["organism"]["lineage"]:
     kd = "Fungi"
  elif "Viridiplantae" in entry["organism"]["lineage"]:
    kd = "Viridiplantae"
  elif "Metazoa" in entry["organism"]["lineage"]:
    kd = "Metazoa"
  else:
    kd = "Other"
  return kd

In [ ]:
ok_entries = []
for response, total in get_batch(url):
    data = response.json()
    for i in range(len(data['results'])):
      entry = data['results'][i]
      if data['results'][i]['features'][0]['location']['end']['value'] > 13 and data['results'][i]['features'][0]['description'] == '':
        ok_entries.append(entry)
print('The total number of entries in the positive set is: ', len(ok_entries))

The total number of entries in the positive set is:  2932


In [ ]:
with open("positive.tsv", 'w') as output_file, open("positive.fasta", "w") as fasta_outputfile:
  for entry in ok_entries:
    line = f'''{entry["primaryAccession"]}{'\t'}{entry["organism"]["scientificName"]}{'\t'}{get_kingdom(entry)}{'\t'}{entry["sequence"]["length"]}{'\t'}{entry["features"][0]["location"]["end"]["value"]}'''
    output_file.write(line + '\n')
    header = f""">{entry["primaryAccession"]} {entry["organism"]["scientificName"]}"""
    sequence = entry["sequence"]["value"]
    fasta_outputfile.write(header + "\n" + sequence + "\n")

# Negatives

In [ ]:
urlo_neg = "https://rest.uniprot.org/uniprotkb/search?format=json&query=%28%28fragment%3Afalse%29+AND+%28taxonomy_id%3A2759%29+AND+%28length%3A%5B40+TO+*%5D%29+AND+%28reviewed%3Atrue%29+AND+%28existence%3A1%29+NOT+%28ft_signal%3A*%29+OR+%28cc_scl_term_exp%3ASL-0091%29+OR+%28cc_scl_term_exp%3ASL-0191%29+OR+%28cc_scl_term_exp%3ASL-0173%29+OR+%28cc_scl_term_exp%3ASL-0209%29+OR+%28cc_scl_term_exp%3ASL-0204%29+OR+%28cc_scl_term_exp%3ASL-0039%29%29&size=500"
response_neg = requests.get(urlo_neg)
session = requests.Session()

In [ ]:
neg_entries = []
count = 0
for response, total in get_batch(urlo_neg):
    data_neg = response.json()
    for i in range(len(data_neg['results'])):
      entry_neg = data_neg['results'][i]
      tm_present = False  # Initialize the boolean flag for this entry
      # Iterate through the features list for each entry
      for feature in entry_neg.get('features', []):
          # Check if the 'type' key exists and is 'Transmembrane'
          if feature.get('type') == 'Transmembrane':
              if feature.get("description") == "Helical":
                if feature.get("location", {}).get("start", {}).get("value", float('inf')) <= 90:
                  tm_present = True
                  count += 1
                  break # Found the specific TM domain, no need to check further features for this entry

      # Append the entry data along with the tm_present flag
      # This ensures that the boolean corresponds to the entry
      neg_entries.append((entry_neg, tm_present))
print('The number of proteins having the transmembrane helix in the first 90 residues is: ', count)
print('The total number of entries in the negative set is: ', len(neg_entries))

The number of proteins having the transmembrane helix in the first 90 residues is:  1384
The total number of entries in the negative set is:  20615


In [ ]:
with open("negative.tsv", 'w') as output_file, open("negative.fasta", "w") as fasta_outputfile:
  for i, entry in enumerate(neg_entries):
    line = f'''{entry[0]["primaryAccession"]}{'\t'}{entry[0]["organism"]["scientificName"]}{'\t'}{get_kingdom(entry[0])}{'\t'}{entry[0]["sequence"]["length"]}{'\t'}{entry[1]}'''
    output_file.write(line + '\n')
    header = f""">{entry[0]["primaryAccession"]} {entry[0]["organism"]["scientificName"]}"""
    sequence = entry[0]["sequence"]["value"]
    fasta_outputfile.write(header + "\n" + sequence + "\n")